# **ETL**

In [1]:
# Main libraries

import requests 
import os 
import pandas as pd  
import numpy as np
from sqlalchemy import create_engine

In [2]:
pd.set_option('display.max_rows', 5)

### **States**
**Method**: csv file import

In [3]:
# File: Main cities in the world
dir_world_cities = os.getcwd()+'\\..\\source_data\\World_cities.csv'

In [4]:
# Data exploration
world_cities = pd.read_csv(dir_world_cities)
world_cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
...,...,...,...,...,...,...,...,...,...,...,...
47866,Hongseong,Hongseong,36.6009,126.6650,"Korea, South",KR,KOR,Chungnam,admin,NaN,1410822139
47867,Charlotte Amalie,Charlotte Amalie,18.3420,-64.9331,U.S. Virgin Islands,VI,VIR,Virgin Islands,primary,NaN,1850037473


In [7]:
# Data transformation
world_cities = world_cities[world_cities['country']=='Mexico']
world_cities = world_cities[['admin_name']]
world_cities = world_cities.rename(columns={'admin_name':'name'})
world_cities = world_cities.sort_values(by='name')
world_cities = world_cities.drop_duplicates(subset='name')
world_cities = world_cities.reset_index(drop =True)
world_cities


,name
0,Aguascalientes
1,Baja California
...,...
30,Yucatán
31,Zacatecas


In [8]:
# File: Population density by state 
dir_density = os.getcwd()+'\\..\\source_data\\Population_density_mx.csv'

In [9]:
# Data exploration
density = pd.read_csv(dir_density, encoding='ANSI')
density

,Estado,Hab. Por km cuadrado
0,Aguascalientes,253.9
1,Baja California,52.8
...,...,...
30,Yucatán,58.7
31,Zacatecas,21.5


In [10]:
# Data transformation
density = density.sort_values(by='Estado')
density= density.rename(columns={'Estado':'name','Hab. Por km cuadrado':'inh./km2'})
density = density.reset_index(drop=True)
density

,name,inh./km2
0,Aguascalientes,253.9
1,Baja California,52.8
...,...,...
30,Yucatán,58.7
31,Zacatecas,21.5


In [11]:
# Exploration of consistency between both tables
comparison =world_cities['name']==density['name']
comparison.name = 'values'
comparison = comparison.to_frame()
comparison = comparison[comparison['values']==False]
comparison

,values
7,False
14,False
29,False


In [12]:
merge = pd.merge(world_cities, density, left_index=True, right_index=True)
merge.iloc[comparison.index] 

,name_x,name_y,inh./km2
7,Coahuila,Coahuila de Zaragoza,20.8
14,Michoacán,Michoacán de Ocampo,81.0
29,Veracruz,Veracruz de Ignacio de la Llave,112.3


In [13]:
density['name']=world_cities['name']
merge = pd.merge(world_cities, density, left_index=True, right_index=True)
merge.iloc[comparison.index] 

,name_x,name_y,inh./km2
7,Coahuila,Coahuila,20.8
14,Michoacán,Michoacán,81.0
29,Veracruz,Veracruz,112.3


In [14]:
# Table of states
states = density
states =states.rename(columns={'inh./km2':'density'})


In [15]:
states

,name,density
0,Aguascalientes,253.9
1,Baja California,52.8
...,...,...
30,Yucatán,58.7
31,Zacatecas,21.5


In [16]:
# Saving the database
file_path =  os.getcwd() + '/../cleaned_data/states.csv'
states.to_csv(file_path, index=False)

In [ ]:
# Database Connection: We use sql server with a trusted connection and weather as database name
engine = create_engine('mssql+pyodbc://SERVER_NAME/weather?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')


In [138]:
# Database load
states.to_sql('states', engine, if_exists='append', index=False)


32